In [72]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [73]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [76]:
def getInfo(filesGrp, img_path):

  dslabel = {
    "id":"",
    "11originalFileJPG":"",
    "11originalFileGrp":[],
    "GeoTagInfo":{
      "coords":[],
      "datetime_original":"",
      "gps_altitude":0,
      "gps_altitude_ref":""    
    },
    "label":[]
  }

# Remove Drive letter with root path
  dslabel['11originalFileJPG'] = img_path[3:]
  dslabel['11originalFileGrp'] = [x[3:] for x in filesGrp  if ('.JPG' not in x)]

  with open(img_path, 'rb') as src:
    img = exifimg(src)

  coords = [
    decimal_coords(img.gps_longitude,
    img.gps_longitude_ref),
      decimal_coords(img.gps_latitude,
    img.gps_latitude_ref)
  ]

  dslabel['id'] = img.datetime_original.replace(" ","").replace(":","") + "_{:.10f}".format(coords[0]).replace(".","") + "_{:.10f}".format(coords[1]).replace(".","") + "_{:f}".format(img.gps_altitude).replace(".","")
  dslabel['GeoTagInfo']['coords'] = coords
  dslabel['GeoTagInfo']['datetime_original'] = img.datetime_original
  dslabel['GeoTagInfo']['gps_altitude'] = img.gps_altitude
  dslabel['GeoTagInfo']['gps_altitude_ref'] = img.gps_altitude_ref
  
  return dslabel

In [86]:
src_folder = r'D:\G04_오너스'
out_folder = r'D:\ToAWS2'

course_list = ['청도','포항','중문','시흥','거창','속리산','춘천','오너스']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
  '시흥':'MGC004',
  '거창':'MGC005',
  '속리산':'MGC006',
  '춘천':'MGC007',
  '오너스':'MGC007'
}

folder_type = ['rgb','ndvi','thumb']

In [87]:
course_id = course_ids.get('춘천')
course_id

'MGC007'

In [88]:
files = glob.glob(os.path.join(src_folder,'**/*.JPG'), recursive=True)
files = [x for x in files if 'DJI' in x]

len(files)

5

In [89]:
files[0]

'D:\\G04_오너스\\CU_근접촬영\\05_병해잔디\\83_(패취류)섬머패취\\G04_CU_0583_DJI_20230826133939_0001_D.JPG'

In [90]:
file_ = files[0]
path = os.path.normpath(file_)
info = path.split(os.sep)
info

['D:',
 'G04_오너스',
 'CU_근접촬영',
 '05_병해잔디',
 '83_(패취류)섬머패취',
 'G04_CU_0583_DJI_20230826133939_0001_D.JPG']

In [91]:
os.path.join(os.path.split(file_)[0],(('_').join(os.path.split(file_)[1].split('_')[:-2])[:-3]+"***_" + os.path.split(file_)[1].split('_')[-2]) + '*')

'D:\\G04_오너스\\CU_근접촬영\\05_병해잔디\\83_(패취류)섬머패취\\G04_CU_0583_DJI_20230826133***_0001*'

In [92]:
glob.glob(os.path.join(os.path.split(file_)[0],(('_').join(os.path.split(file_)[1].split('_')[:-2])[:-3]+"***_" + os.path.split(file_)[1].split('_')[-2]) + '*'), recursive=True)

['D:\\G04_오너스\\CU_근접촬영\\05_병해잔디\\83_(패취류)섬머패취\\G04_CU_0583_DJI_20230826133939_0001_D.JPG',
 'D:\\G04_오너스\\CU_근접촬영\\05_병해잔디\\83_(패취류)섬머패취\\G04_CU_0583_DJI_20230826133939_0001_MS_G.TIF',
 'D:\\G04_오너스\\CU_근접촬영\\05_병해잔디\\83_(패취류)섬머패취\\G04_CU_0583_DJI_20230826133939_0001_MS_NIR.TIF',
 'D:\\G04_오너스\\CU_근접촬영\\05_병해잔디\\83_(패취류)섬머패취\\G04_CU_0583_DJI_20230826133939_0001_MS_R.TIF',
 'D:\\G04_오너스\\CU_근접촬영\\05_병해잔디\\83_(패취류)섬머패취\\G04_CU_0583_DJI_20230826133939_0001_MS_RE.TIF']

In [93]:
InfoJsonArray = []
target_folder = []

for idx, file_ in enumerate(files):

  course = list(filter(lambda y: y in file_, course_list))[0]

  path = os.path.normpath(file_)
  info = path.split(os.sep)

  filesGrp = glob.glob(os.path.join(os.path.split(file_)[0],(('_').join(os.path.split(file_)[1].split('_')[:-2])[:-3]+"***_" + os.path.split(file_)[1].split('_')[-2]) + '*'), recursive=True)
  print(file_)
  file_info = getInfo(filesGrp, file_)

  file_info['label'] = {
    "level1":info[-3],
    "level2":info[-2],
    "level3":"",
    "TurfType":""
  }

  InfoJsonArray.append( file_info)
  
#   print(save_name)

#   print('{} out of {} files Completed'.format(idx, len(target_files)))

save_name = os.path.join(out_folder, 'afterLabeldata'+course_id+'.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(InfoJsonArray, final , ensure_ascii=False)



  

D:\G04_오너스\CU_근접촬영\05_병해잔디\83_(패취류)섬머패취\G04_CU_0583_DJI_20230826133939_0001_D.JPG
D:\G04_오너스\CU_근접촬영\05_병해잔디\83_(패취류)섬머패취\G04_CU_0583_DJI_20230826133941_0002_D.JPG
D:\G04_오너스\CU_근접촬영\05_병해잔디\83_(패취류)섬머패취\G04_CU_0583_DJI_20230826133943_0003_D.JPG
D:\G04_오너스\CU_근접촬영\05_병해잔디\83_(패취류)섬머패취\G04_CU_0583_DJI_20230826133945_0004_D.JPG
D:\G04_오너스\CU_근접촬영\05_병해잔디\83_(패취류)섬머패취\G04_CU_0583_DJI_20230826133947_0005_D.JPG


In [94]:
InfoJsonArray

[{'id': '20230826133939_1276601483056_377679769444_204559000',
  '11originalFileJPG': 'G04_오너스\\CU_근접촬영\\05_병해잔디\\83_(패취류)섬머패취\\G04_CU_0583_DJI_20230826133939_0001_D.JPG',
  '11originalFileGrp': ['G04_오너스\\CU_근접촬영\\05_병해잔디\\83_(패취류)섬머패취\\G04_CU_0583_DJI_20230826133939_0001_MS_G.TIF',
   'G04_오너스\\CU_근접촬영\\05_병해잔디\\83_(패취류)섬머패취\\G04_CU_0583_DJI_20230826133939_0001_MS_NIR.TIF',
   'G04_오너스\\CU_근접촬영\\05_병해잔디\\83_(패취류)섬머패취\\G04_CU_0583_DJI_20230826133939_0001_MS_R.TIF',
   'G04_오너스\\CU_근접촬영\\05_병해잔디\\83_(패취류)섬머패취\\G04_CU_0583_DJI_20230826133939_0001_MS_RE.TIF'],
  'GeoTagInfo': {'coords': [127.66014830555557, 37.76797694444444],
   'datetime_original': '2023:08:26 13:39:39',
   'gps_altitude': 204.559,
   'gps_altitude_ref': <GpsAltitudeRef.ABOVE_SEA_LEVEL: 0>},
  'label': {'level1': '05_병해잔디',
   'level2': '83_(패취류)섬머패취',
   'level3': '',
   'TurfType': ''}},
 {'id': '20230826133941_1276601509167_377679778611_204499000',
  '11originalFileJPG': 'G04_오너스\\CU_근접촬영\\05_병해잔디\\83_(패취류)섬머패취\\G04

Gather all data json files from Output Folder

In [52]:
out_folder = r'D:\ToAWS'

In [56]:
files = glob.glob(os.path.join(out_folder,'**/*.json'), recursive=True)
files = [x for x in files if 'photo' in x]
files

['D:\\ToAWS\\MGC001\\20230811\\photo.json',
 'D:\\ToAWS\\MGC002\\20230823\\photo.json']

In [57]:
file_ = files[1]

In [58]:
json_dsDB = json.load(open(file_, encoding="utf8"))

In [67]:
[x for x in json_dsDB if '20230823162127' in x['id']]

[{'id': '20230823162127_1286447972500_356633918611_161213000',
  'originalFileJPG': 'original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\10_스타트퍼팅그린\\2023_0823_청도그레이스_스타트퍼팅그린_1.5m\\20230823_1621-1630_엽고성병,달라스팟,벤트,그린\\DJI_20230823162127_0011_D.JPG',
  'originalFileGrp': ['original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\10_스타트퍼팅그린\\2023_0823_청도그레이스_스타트퍼팅그린_1.5m\\20230823_1621-1630_엽고성병,달라스팟,벤트,그린\\DJI_20230823162127_0011_MS_G.TIF',
   'original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\10_스타트퍼팅그린\\2023_0823_청도그레이스_스타트퍼팅그린_1.5m\\20230823_1621-1630_엽고성병,달라스팟,벤트,그린\\DJI_20230823162127_0011_MS_NIR.TIF',
   'original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\10_스타트퍼팅그린\\2023_0823_청도그레이스_스타트퍼팅그린_1.5m\\20230823_1621-1630_엽고성병,달라스팟,벤트,그린\\DJI_20230823162127_0011_MS_R.TIF',
   'original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\10_스타트퍼팅그린\\2023_0823_청도그레이스_스타트퍼팅그린_1.5m\\20230823_1621-1630_엽고성병,달라스팟,벤트,그린\\DJI_20230823162127_0011_MS_RE.TIF'],
  'GeoTagInfo': {'coords': [128.64479724999998, 35.66339186111111],
   'datetim

In [71]:
InfoJsonArray[100]

{'id': '20230823162145_1286447236944_356634455000_160026000',
 '11originalFileJPG': 'G07_청도그레이스\\CU_근접촬영\\05_병해잔디\\86_(조류)남조류\\G07_CU_0586_DJI_20230823162145_0014_D.JPG',
 '11originalFileGrp': ['G07_청도그레이스\\CU_근접촬영\\05_병해잔디\\86_(조류)남조류\\G07_CU_0586_DJI_20230823162145_0014_MS_G.TIF',
  'G07_청도그레이스\\CU_근접촬영\\05_병해잔디\\86_(조류)남조류\\G07_CU_0586_DJI_20230823162145_0014_MS_NIR.TIF',
  'G07_청도그레이스\\CU_근접촬영\\05_병해잔디\\86_(조류)남조류\\G07_CU_0586_DJI_20230823162145_0014_MS_R.TIF',
  'G07_청도그레이스\\CU_근접촬영\\05_병해잔디\\86_(조류)남조류\\G07_CU_0586_DJI_20230823162145_0014_MS_RE.TIF'],
 'GeoTagInfo': {'coords': [128.64472369444442, 35.6634455],
  'datetime_original': '2023:08:23 16:21:45',
  'gps_altitude': 160.026,
  'gps_altitude_ref': <GpsAltitudeRef.ABOVE_SEA_LEVEL: 0>},
 'label': {'level1': '05_병해잔디',
  'level2': '86_(조류)남조류',
  'level3': '',
  'TurfType': ''}}

In [ ]:
20230823162127_1286447972500_356633918611_161213000